#Risk stats
The table below shows expected outcomes for large battles in risk.  The row corresponds to the size of the attacking army, the column corresponds to the size of the defending army.

The greener a cell is, the more likely it is the attacker can take control of a territory.  The redder a cell is, the more likely it is the defender will maintain control of a territory.

Each cell contains a number corresponding to an expected outcome.  If the number is positive, it represents the number of armies the attacker will likely be left with.  If the number is negative, it represents the number of armies the defender will likely be left with.  

##Details
For a primer on the rules of risk: http://media.wizards.com/2015/downloads/ah/Risk_rules.pdf
Each configuration was simulated 5,000 times to determine the frequency of outcomes.  

In [1]:
from collections import namedtuple
import random
rand = random.Random()

Armies = namedtuple("Armies", ["attackers", "defenders"])

class Side:
    ATTACKER = 1
    DEFENDER = 2

def eval_roll(attacker_dice, defender_dice):
    "Return the delta for each side... (delta_attackers, delta_defenders)"
    attacker_wins, defender_wins = 0, 0
    for attacker_roll, defender_roll in zip(sorted(attacker_dice)[::-1], sorted(defender_dice)[::-1]):
        if attacker_roll > defender_roll:
            attacker_wins += 1
        else:  #defender wins tie
            defender_wins += 1
    return Armies(-defender_wins, -attacker_wins)

assert(eval_roll([5, 4, 1], [5, 1]) == Armies(-1, -1))
assert(eval_roll([6, 6, 6], [6]) == Armies(-1, 0))

def roll_dice(num):
    return [rand.randint(1, 6) for i in range(num)]

def run_battle(armies):
    attacking, defending = armies
    while attacking > 1 and defending > 0:
        delta_attacking, delta_defending = eval_roll(
            roll_dice(min(attacking - 1, 3)),
            roll_dice(min(defending, 2)))
        attacking += delta_attacking
        defending += delta_defending
    return Armies(attacking, defending)

run_battle(Armies(6, 5))

Armies(attackers=1, defenders=5)

In [47]:
def get_winner(armies):
    if armies.defenders == 0:
        return Side.ATTACKER
    else:
        return Side.DEFENDER

def run_many_battles(armies, num_iterations=2500):
    "Given a starting configuration of |armies|, simulate |num_iterationgs| battles and return stats on the results"
    attacker_wins, defender_wins = 0, 0
    attacker_remainings, defender_remainings = [], []
    for i in range(num_iterations):
        res = run_battle(armies)
        if get_winner(res) == Side.ATTACKER:
            attacker_wins += 1
            attacker_remainings.append(res.attackers)
        else:
            defender_wins += 1
            defender_remainings.append(res.defenders)
    
    attacker_mean = 0.0
    defender_mean = 0.0
    
    all_mean = sum(attacker_remainings + [-x for x in defender_remainings]) / len(attacker_remainings + defender_remainings)
    try:
        #attacker_mean = max(set(attacker_remainings), key=attacker_remainings.count)
        attacker_mean = sum(attacker_remainings) / len(attacker_remainings)
    except:
        pass
    
    try:
        #defender_mean = max(set(defender_remainings), key=defender_remainings.count)
        defender_mean = sum(defender_remainings) / len(defender_remainings)
    except:
        pass
            
    return (attacker_wins / num_iterations, defender_wins / num_iterations, attacker_mean, defender_mean, all_mean)

print(run_many_battles(Armies(0, 0)))

(1.0, 0.0, 0.0, 0.0, 0.0)


In [ ]:
from IPython.display import HTML, display

def generate_battle_matrix(max_attackers=20, max_defenders=20):
    "Call run_many_battles on various starting configurations"
    matrix = [[0 for x in range(max_defenders)] for y in range(max_attackers)]
    for attackers in range(max_defenders):
        for defenders in range(max_defenders):
            matrix[attackers][defenders] = run_many_battles(Armies(attackers, defenders))
    return matrix

def colorize(res, attackers_start, defenders_start):
    "Assign a color to the stats collected by run_many_battles"
    rate, _, attackers_left, defenders_left, all_left = res
    
    """
    if rate > 0.5:
        whiteness = int(255 * (1 - (attackers_left / attackers_start)**2.0))
        return "#%02xff%02x" % (whiteness, whiteness)
    else:
        whiteness = int(255 * (1 - (defenders_left / defenders_start)**2.0))
        return "#ff%02x%02x" % (whiteness, whiteness)
    """ 
    whiteness = int(255 * 2 * ((0.5 - abs(rate - 0.5))**0.8))
    if rate > 0.5:
        return "#%02xff%02x" % (whiteness, whiteness)
    else:
        return "#ff%02x%02x" % (whiteness, whiteness)

def render_matrix_html(matrix):
    "Render the results of generate_battle_matrix as an html table"
    rows = []
    
    first_row = ["<th></th>"]
    for num_defenders, _ in enumerate(matrix[0]):
        if num_defenders == 0:
            continue
        first_row.append("<th>d=%s</th>" % (num_defenders))
    rows.append(''.join(first_row))
    
    for num_attackers, stats in enumerate(matrix):
        if num_attackers == 0:
            continue
            
        cells = ["<th>a=%s</th>" % (num_attackers)]
        for num_defenders, res in enumerate(stats):
            if num_defenders == 0:
                continue
                
            successRate = res[0]
            remainingArmies = 0
            if successRate >= 0.5:
                remainingArmies = "+%2.1f" % res[2]
            else:
                remainingArmies = "-%2.1f" % res[3]
            color = colorize(res, num_attackers, num_defenders)
            remainingArmies = "%2.1f" % res[4]
            cells.append("<td style='background-color: %s'>%s</td>" % (color, remainingArmies))
        rows.append(''.join(cells))
        
    rows = ["<tr>%s</tr>" % (row) for row in rows]
    
    return HTML("<table>%s</table>" % (''.join(rows)))


display(render_matrix_html(generate_battle_matrix(25, 25)))